In [2]:
%reload_ext autoreload
%autoreload 2

In [4]:
import utils 
utils.init_logging()

import polars as pl
from polars import col, lit
import altair as alt
import simrs
from scenarios import builder as b
import scenarios
import cli
import analysis
from asgiref.sync import async_to_sync

In [12]:
from replay_cache import Replay
import scenarios.examples as ex

def barbell_replay(ticks=100, num_agents=10):
    # Agents
    agent_ids = ["a" + str(i) for i in range(1, num_agents + 1)]

    # Goods
    wheat = "Wheat"
    goods = [wheat]

    # Ports
    la = "la"
    lb = "lb"
    m1 = "m1"
    m2 = "m2"
    m3 = "m3"
    ra = "ra"

    # la
    #     m1 m2 m3 ra
    # lb
    edges = [
        (la, lb),
        (la, m1),
        (lb, m1),
        (m1, m2),
        (m2, m3),
        (m3, ra),
    ]
    _market = lambda c, s: b._market(
        wheat,
        b._market_info(
            production=100,
            consumption=100 + c,
            supply=1000 - s,
            pricer=b._inversep(100_000),
        ),
    )

    # net balance
    ports = [
        b._port(la, _market(0, 20)),
        b._port(lb, _market(0, 20)),
        b._port(m1, _market(0, 20)),
        b._port(m2, _market(0, 0)),
        b._port(m3, _market(0, 0)),
        b._port(ra, _market(0, -20)),
    ]

    _agent = lambda id, pos: b._agent(id, pos, 1000, "Exhaustive")
    agents = [
        _agent(id, port_id) for (id, port_id) in zip(agent_ids, [la] * num_agents)
    ]

    return b._inputFormat(
        agents=agents, ports=ports, edges=edges, opts=b._opts(ticks=ticks)
    )


b.save_scenario(barbell_replay(10, 1), "long")